## Evaluating Patterns in Alphafold Protein Prediction Confidence 

#### Attempt to start Pyspark

In [1]:
import os
import pyspark
conf = pyspark.SparkConf()
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
#conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','32')

sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)

Error occurred during initialization of VM
Too small maximum heap


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [ ]:
# imports
import os
from pyspark.sql.functions import col, lit, length
from  pyspark.sql.functions import input_file_name
from pyspark.sql.functions import split, substring_index, regexp_replace
from pyspark.sql.functions import collect_list
from pyspark.sql.window import Window
from pyspark.sql.functions import mean, size, udf
from pyspark.sql.types import FloatType, IntegerType, StringType
from collections import Counter
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType
from pyspark.sql.functions import array
import numpy as np
from pyspark.sql.types import MapType, StringType, FloatType

os.getcwd()

### Load in as df

In [2]:
import pandas as pd 

alpha_df = pd.read_csv('alpha_df.csv')
alpha_df.head()

,protein_id,protein_seq,protein_seq_len,most_common_aa,aa_pLDDT_list,avg_pLDDT,ALA_ratio,ARG_ratio,ASN_ratio,ASP_ratio,...,LEU_ratio,LYS_ratio,MET_ratio,PHE_ratio,PRO_ratio,SER_ratio,THR_ratio,TRP_ratio,TYR_ratio,VAL_ratio
0,P01764,"['MET', 'GLU', 'PHE', 'GLY', 'LEU', 'SER', 'TR...",117,SER,"[41.81999969482422, 45.2599983215332, 54.70999...",91.027010,0.085470,0.042735,0.025641,0.025641,...,0.102564,0.042735,0.025641,0.042735,0.017094,0.136752,0.042735,0.025641,0.051282,0.076923
1,P01833,"['MET', 'LEU', 'LEU', 'PHE', 'VAL', 'LEU', 'TH...",764,SER,"[52.939998626708984, 50.970001220703125, 50.18...",76.677880,0.075916,0.053665,0.051047,0.049738,...,0.086387,0.054974,0.005236,0.032723,0.041885,0.089005,0.057592,0.014398,0.032723,0.085079
2,P02545,"['MET', 'GLU', 'THR', 'PRO', 'SER', 'GLN', 'AR...",664,LEU,"[35.939998626708984, 32.06999969482422, 36.529...",76.273735,0.088855,0.096386,0.031627,0.054217,...,0.109940,0.060241,0.015060,0.012048,0.022590,0.103916,0.054217,0.006024,0.016566,0.046687
3,P02656,"['MET', 'GLN', 'PRO', 'ARG', 'VAL', 'LEU', 'LE...",99,ALA,"[53.900001525878906, 69.83000183105469, 77.410...",68.228790,0.151515,0.040404,0.000000,0.070707,...,0.111111,0.060606,0.030303,0.040404,0.030303,0.121212,0.050505,0.030303,0.020202,0.090909
4,P02679,"['MET', 'SER', 'TRP', 'SER', 'LEU', 'HIS', 'PR...",453,LEU,"[36.93000030517578, 34.45000076293945, 38.4000...",85.611370,0.061810,0.026490,0.052980,0.075055,...,0.075055,0.075055,0.019868,0.044150,0.033113,0.066225,0.066225,0.024283,0.052980,0.033113


In [3]:
# The 20 Amino Acids Found in Proteins.

# Define a list of all unique amino acids (considering standard amino acids)
amino_acids = ["ALA", "ARG", "ASN", "ASP", "CYS", "GLN", "GLU", "GLY", "HIS",
               "ILE", "LEU", "LYS", "MET", "PHE", "PRO", "SER", "THR", "TRP", "TYR", "VAL"]

In [4]:
cols_to_keep = ["avg_pLDDT", "protein_seq_len"] + [aa + "_ratio" for aa in amino_acids]
print(cols_to_keep)

['avg_pLDDT', 'protein_seq_len', 'ALA_ratio', 'ARG_ratio', 'ASN_ratio', 'ASP_ratio', 'CYS_ratio', 'GLN_ratio', 'GLU_ratio', 'GLY_ratio', 'HIS_ratio', 'ILE_ratio', 'LEU_ratio', 'LYS_ratio', 'MET_ratio', 'PHE_ratio', 'PRO_ratio', 'SER_ratio', 'THR_ratio', 'TRP_ratio', 'TYR_ratio', 'VAL_ratio']


In [5]:
alpha_df_subset = alpha_df[cols_to_keep]

alpha_df_subset.head()

,avg_pLDDT,protein_seq_len,ALA_ratio,ARG_ratio,ASN_ratio,ASP_ratio,CYS_ratio,GLN_ratio,GLU_ratio,GLY_ratio,...,LEU_ratio,LYS_ratio,MET_ratio,PHE_ratio,PRO_ratio,SER_ratio,THR_ratio,TRP_ratio,TYR_ratio,VAL_ratio
0,91.027010,117,0.085470,0.042735,0.025641,0.025641,0.025641,0.042735,0.042735,0.119658,...,0.102564,0.042735,0.025641,0.042735,0.017094,0.136752,0.042735,0.025641,0.051282,0.076923
1,76.677880,764,0.075916,0.053665,0.051047,0.049738,0.027487,0.040576,0.070681,0.087696,...,0.086387,0.054974,0.005236,0.032723,0.041885,0.089005,0.057592,0.014398,0.032723,0.085079
2,76.273735,664,0.088855,0.096386,0.031627,0.054217,0.007530,0.060241,0.106928,0.063253,...,0.109940,0.060241,0.015060,0.012048,0.022590,0.103916,0.054217,0.006024,0.016566,0.046687
3,68.228790,99,0.151515,0.040404,0.000000,0.070707,0.000000,0.060606,0.050505,0.030303,...,0.111111,0.060606,0.030303,0.040404,0.030303,0.121212,0.050505,0.030303,0.020202,0.090909
4,85.611370,453,0.061810,0.026490,0.052980,0.075055,0.024283,0.052980,0.057395,0.075055,...,0.075055,0.075055,0.019868,0.044150,0.033113,0.066225,0.066225,0.024283,0.052980,0.033113


In [6]:
# Compute the correlation matrix for the specified columns
correlation_matrix = alpha_df_subset.corr()

# Extract the correlation of avg_pLDDT with each amino acid ratio
correlation_with_avg_pLDDT = correlation_matrix.loc["avg_pLDDT", [aa + "_ratio" for aa in amino_acids]]

# Display the correlation of avg_pLDDT with each amino acid ratio
print("Correlation between avg_pLDDT and each amino acid ratio:")
print(correlation_with_avg_pLDDT)

Correlation between avg_pLDDT and each amino acid ratio:
ALA_ratio    0.099032
ARG_ratio   -0.139749
ASN_ratio    0.189308
ASP_ratio    0.245841
CYS_ratio   -0.270056
GLN_ratio   -0.131247
GLU_ratio   -0.025427
GLY_ratio   -0.278491
HIS_ratio   -0.100018
ILE_ratio    0.452081
LEU_ratio    0.321871
LYS_ratio    0.098032
MET_ratio    0.282805
PHE_ratio    0.296939
PRO_ratio   -0.392591
SER_ratio   -0.514643
THR_ratio    0.116329
TRP_ratio    0.166481
TYR_ratio   -0.080001
VAL_ratio    0.466377
Name: avg_pLDDT, dtype: float64


In [7]:
from scipy.stats import pearsonr

# Initialize dictionaries to hold correlation coefficients and p-values
correlation_dict = {}
p_value_dict = {}

# Compute the Pearson correlation and p-value for each amino acid ratio with avg_pLDDT
for aa in amino_acids:
    # Get the amino acid ratio column and avg_pLDDT column
    aa_ratio_col = alpha_df_subset[aa + "_ratio"]
    avg_pLDDT_col = alpha_df_subset["avg_pLDDT"]

    # Compute the Pearson correlation and p-value
    correlation, p_value = pearsonr(aa_ratio_col, avg_pLDDT_col)
    
    # Store the correlation coefficient and p-value in dictionaries
    correlation_dict[aa] = correlation
    p_value_dict[aa] = p_value

# Convert the dictionaries to Pandas DataFrames for easy viewing
correlation_df = pd.DataFrame(list(correlation_dict.items()), columns=["Amino Acid", "Correlation Coefficient"])
p_value_df = pd.DataFrame(list(p_value_dict.items()), columns=["Amino Acid", "p-value"])

# Display the correlation coefficients and p-values
print("Correlation coefficients:")
print(correlation_df)
print("\nP-values:")
print(p_value_df)

Correlation coefficients:
   Amino Acid  Correlation Coefficient
0         ALA                 0.099032
1         ARG                -0.139749
2         ASN                 0.189308
3         ASP                 0.245841
4         CYS                -0.270056
5         GLN                -0.131247
6         GLU                -0.025427
7         GLY                -0.278491
8         HIS                -0.100018
9         ILE                 0.452081
10        LEU                 0.321871
11        LYS                 0.098032
12        MET                 0.282805
13        PHE                 0.296939
14        PRO                -0.392591
15        SER                -0.514643
16        THR                 0.116329
17        TRP                 0.166481
18        TYR                -0.080001
19        VAL                 0.466377

P-values:
   Amino Acid       p-value
0         ALA  1.341611e-03
1         ARG  5.716280e-06
2         ASN  6.779652e-10
3         ASP  7.291010e-16
4   

In [8]:
# Compute the correlation matrix for the specified columns
correlation_matrix = alpha_df_subset.corr()

# Extract the correlation coefficient between protein_seq_len and avg_pLDDT
correlation_protein_len_avg_pLDDT = correlation_matrix.loc["protein_seq_len", "avg_pLDDT"]

# Display the correlation coefficient
print(f"Correlation between protein sequence length and avg_pLDDT: {correlation_protein_len_avg_pLDDT}")

Correlation between protein sequence length and avg_pLDDT: 0.009101614834234825


In [9]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Separate the target (avg_pLDDT) and the features (all other columns)
target = alpha_df_subset['avg_pLDDT']
features = alpha_df_subset.drop('avg_pLDDT', axis=1)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create and train the LinearRegression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model's performance
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Display the evaluation metrics
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R^2 Score: {r2}")


Root Mean Squared Error (RMSE): 9.797781075397399
Mean Absolute Error (MAE): 7.710003390059241
R^2 Score: 0.5406109509749809


In [10]:
import matplotlib.pyplot as plt

# Make predictions on the training and test data
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Create a figure for plotting
plt.figure(figsize=(8, 8))

# Plot the actual vs. predicted values for the training set
plt.scatter(y_train, y_train_pred, color='blue', alpha=0.6, label='Train')

# Plot the actual vs. predicted values for the test set
plt.scatter(y_test, y_test_pred, color='red', alpha=0.6, label='Test')

# Add a diagonal line for reference (where actual equals predicted)
plt.plot([alpha_df_subset['avg_pLDDT'].min(), alpha_df_subset['avg_pLDDT'].max()],
         [alpha_df_subset['avg_pLDDT'].min(), alpha_df_subset['avg_pLDDT'].max()],
         color='gray', linestyle='--')

# Add title and labels
plt.title('Actual vs. Predicted avg_pLDDT')
plt.xlabel('Actual avg_pLDDT')
plt.ylabel('Predicted avg_pLDDT')

# Add a legend to the plot
plt.legend()

# Show the plot
plt.show()

ModuleNotFoundError: No module named 'matplotlib'